## This Jupyter Notebook can be used for detecting and cropping faces from both live and spoof classes. 
 In case dlib's face detector fails to detect a face in the video, Haar Cascades will be applied.

In [ ]:
import os
import cv2
import dlib

In [ ]:
# Define the input and output directories 
input_directory = "/u/45/demo_file/long/attack"
output_directory = "/u/45/demo_file/long/test_cropped"

In [ ]:


def detect_and_crop_faces(image, output_folder, file_base_name, scale_factor=1.7, target_size=(224, 224)):
    """
    Detect and crop faces from the input image and save them to the output folder.

    Parameters:
        image (ndarray): The input image from which faces will be detected.
        output_folder (str): Directory to save the cropped face images.
        file_base_name (str): Base name for the output files.
        scale_factor (float): Factor to enlarge the detected face bounding box.
        target_size (tuple): Target size for the cropped faces (default: (224, 224)).
    """
    detector = dlib.get_frontal_face_detector()  # Initialize dlib's face detector
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert the image to grayscale
    faces = detector(gray)  # Detect faces using dlib

    # If no faces are detected with dlib, use Haar Cascades as a fallback
    if len(faces) == 0:
        print(f"No faces detected with dlib, trying Haar Cascades method.")
        haar_cascade_path = 'haarcascade_frontalface_default.xml'  # Update with actual path
        face_cascade = cv2.CascadeClassifier(haar_cascade_path)

        # Detect faces using Haar Cascades
        faces_cv = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(int(30 * scale_factor), int(30 * scale_factor)))
        faces = [dlib.rectangle(x, y, x + w, y + h) for (x, y, w, h) in faces_cv]

    # If still no faces are detected, skip this image
    if len(faces) == 0:
        print(f"No faces detected with both methods.")
        return
    
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over detected faces and crop them
    for i, face in enumerate(faces):
        # Get bounding box coordinates
        (x, y, w, h) = (face.left(), face.top(), face.width(), face.height())
        
        # Enlarge bounding box
        new_x = max(0, x - int((scale_factor - 1) * w / 2))
        new_y = max(0, y - int((scale_factor - 1) * h / 2))
        new_w = int(w * scale_factor)
        new_h = int(h * scale_factor)
        
        # Crop the enlarged face region
        cropped_face = image[new_y:new_y + new_h, new_x:new_x + new_w]

        # Check if the cropped face is valid and not empty
        if cropped_face is not None and cropped_face.size > 0:
            # Resize the cropped face to target size (224x224)
            resized_face = cv2.resize(cropped_face, target_size, interpolation=cv2.INTER_CUBIC)
            face_output_path = os.path.join(output_folder, f"{file_base_name}_face_{i}.jpg")
            cv2.imwrite(face_output_path, resized_face)
            print(f"Face {i} saved as: {face_output_path}")
        else:
            print(f"Error: Cropped face is empty, face {i}")

def extract_central_frame(video_path, output_directory):
    """
    Extract the central frame from a video and detect and crop faces.

    Parameters:
        video_path (str): Path to the video file.
        output_directory (str): Directory to save the cropped face images.
    """
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    central_frame_index = total_frames // 2

    cap.set(cv2.CAP_PROP_POS_FRAMES, central_frame_index)
    ret, frame = cap.read()
    
    if ret:
        video_name = os.path.basename(video_path)
        file_base_name = os.path.splitext(video_name)[0]
        detect_and_crop_faces(frame, output_directory, file_base_name)
    else:
        print(f"Failed to read frame from {video_path}")

    cap.release()

def get_video_files(input_directory):
    """
    Recursively collect video file paths from a given directory.

    Parameters:
        input_directory (str): Directory to search for video files.

    Returns:
        list: A list of paths to video files.
    """
    video_files = []
    for root, dirs, files in os.walk(input_directory):
        for file in files:
            if file.endswith(('.mp4', '.avi', '.mov', '.mkv')):
                video_files.append(os.path.join(root, file))
    return video_files

def process_videos(input_directory, output_directory):
    """
    Process all video files in the input directory to extract faces.

    Parameters:
        input_directory (str): Directory containing video files.
        output_directory (str): Directory to save cropped face images.
    """
    os.makedirs(output_directory, exist_ok=True)  # Create output directory if it doesn't exist
    video_files = get_video_files(input_directory)  # Get the list of video files
    
    # Process each video file to extract and crop faces
    for video_file in video_files:
        extract_central_frame(video_file, output_directory)

# Main block to execute the video processing
if __name__ == "__main__":
    # Process all video files in the input directory
    process_videos(input_directory, output_directory)
